<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#IMPORT" data-toc-modified-id="IMPORT-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>IMPORT</a></span></li><li><span><a href="#FUNCTIONS" data-toc-modified-id="FUNCTIONS-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>FUNCTIONS</a></span></li><li><span><a href="#Suffix-Tree" data-toc-modified-id="Suffix-Tree-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Suffix Tree</a></span><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Creation-of-the-tree" data-toc-modified-id="Creation-of-the-tree-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Creation of the tree</a></span></li><li><span><a href="#Give-node-liste" data-toc-modified-id="Give-node-liste-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Give node liste</a></span></li></ul></li><li><span><a href="#Clustering" data-toc-modified-id="Clustering-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Clustering</a></span><ul class="toc-item"><li><span><a href="#Creation-vector-weight" data-toc-modified-id="Creation-vector-weight-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Creation vector weight</a></span><ul class="toc-item"><li><span><a href="#tf---idf,-frequency–inverse-document-frequency" data-toc-modified-id="tf---idf,-frequency–inverse-document-frequency-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>tf - idf, frequency–inverse document frequency</a></span></li><li><span><a href="#Matrix-distance" data-toc-modified-id="Matrix-distance-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>Matrix distance</a></span></li></ul></li><li><span><a href="#Hierarchical-clustering" data-toc-modified-id="Hierarchical-clustering-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Hierarchical clustering</a></span></li><li><span><a href="#Choose-the-number-of-cluster" data-toc-modified-id="Choose-the-number-of-cluster-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Choose the number of cluster</a></span></li><li><span><a href="#Plot-the-clustering" data-toc-modified-id="Plot-the-clustering-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Plot the clustering</a></span></li></ul></li><li><span><a href="#Show-sequences" data-toc-modified-id="Show-sequences-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Show sequences</a></span></li></ul></div>

# IMPORT

In [1]:

import os
import glob
import sys
import numpy as np
import pandas as pd

import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram, to_tree
from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist as cdist

from suffix_tree import Tree

import sklearn as sk


from sklearn.metrics import pairwise_distances
from tqdm import tqdm
from sklearn.cluster import AffinityPropagation
from numpy.linalg import eig

from sklearn import preprocessing
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

from scipy.cluster.hierarchy import fcluster

from itertools import groupby
import itertools

from sklearn.metrics import silhouette_score

import pickle




In [5]:
cd ../../../../../../

/


# FUNCTIONS

In [21]:
def load_data():
    path_data    ='../data/raw/t5/'
    name_dataset = 't5_during_UAS_TNT.csv'           
    data  = pd.read_csv(path_data + name_dataset, sep=',')
    data.rename(columns={'neuron': 'line', 'exhaustive 6':'time_6', \
                     'compact 6':'seq_6','exhaustive 12':'time_12', \
                         'compact 12':'seq_12' },inplace=True)
    return data

In [3]:
def clean_z_temporal_cut(data, time_cut_s):
    
    dt       = 0.1
    line     = data['line']
    seq_6    = data['time_6'].values
    seq_12   = data['time_12'].values
    
    ## remove z
    index_6    = []
    index_cut  =int( np.round(time_cut_s/dt)+1 )
    
    ## index to remove z
    for i,stuff in enumerate(seq_6):
        if 'z' not in stuff:
            index_6.append(i) 
            
    ### filtre pour avoir moins de point
    seq_6_z   = seq_6[index_6]
    seq_12_z  = seq_12[index_6]
    line_6_z  = line[index_6] 

    ## remove first points 
    index_init = 10
    
    
    for indice,stuff in enumerate(seq_6_z):
        stuff           = stuff[index_init: index_cut]
        seq_6_z[indice] = (''.join(i for i, _ in groupby(stuff)))
    
    for indice,stuff in enumerate(seq_12_z):
        stuff            = stuff[index_init: index_cut]
        seq_12_z[indice] = (''.join(i for i, _ in groupby(stuff)))
        
    data  = pd.DataFrame({'line':line_6_z , 'seq_6' : seq_6_z, 'seq_12': seq_12_z})

    return data 

In [4]:
def show_weight_sequences(data_cluster,  data_out,Vector_weight, nb_dict, mode):
    nb_motif = 15
    tt = ['Clus']
    for i in range(nb_motif):
        tt.append('motif_' + str(i))
    target_len = len(tt)
    table = PrettyTable(tt)
    del tt
    
    all_weight = pd.DataFrame(np.zeros((len(data_cluster),len(Vector_weight.iloc[0]))),index=data_cluster.index,columns=Vector_weight.columns)
    for idx in range(len(data_cluster)):
        all_weight.loc[idx]=list(Vector_weight.loc[data_cluster.loc[idx,'line']].sum(axis=0)/len(data_cluster.loc[idx]['line']))

    for seq in all_weight.columns:

        all_weight.loc['Moy',seq]= np.mean(all_weight[seq].iloc[:-2])    
    All_weight = copy.deepcopy(all_weight)

    nb_tot = len(data_cluster.index)
    mode = 'motif'
    Liste = []
    for k in range(nb_tot):
        weight = All_weight.loc[k].sort_values(ascending=False)
        number_cluster = k
        if nb_dict == 6:
            name_out = 'seq_6'
        else:
            name_out = 'seq_12'
        ttt       = ['clus_' + str(k)]
        local_len =  min(nb_motif,len(all_weight.iloc[0]),nb_motif+1)

        for kk in range(local_len):
            if weight.iloc[kk] > 0.0:
                if mode   == 'weight':
                    ttt.append(weight.index[kk].replace(" ", "") + ' ' + str(weight.iloc[kk])[0:4])
                elif mode == 'motif':
                    ttt.append(weight.index[kk].replace(" ", ""))
            else:
                ttt.append('NA')

        table.add_row(ttt)
        Liste.append(ttt)
    df = pd.DataFrame.from_records(Liste)    
    print(table)
    return df

In [36]:
def give_node_liste(tree,data,Line):

    def f (node):
        """ Compute a nodemap """
        L=[a.split(' ')[0] for a in list(node.compute_C())]
        liste_count=[L.count(l) for l in Line]
        nb_leaf = -1
        if len(set(L))>1:
            if str(node.path).replace(' ', '') in list(data['seq_6'][:]):
                Node.append([node.path,liste_count,(len(set(L)))])

    Node = []
    tree.root.pre_order(f)
    return Node

In [37]:
def hierarchical_clusters(data = 'data', nb_dict=6, matrice_distance='matrice_distance', nb_cluster_max=20):
    
    if nb_dict==6:
        name_col = 'seq_6'
    else:
        name_col = 'seq_12' 
        
    data_out           = data.copy(deep=True)
    data_out           = data_out.groupby('line')[name_col].apply(list).reset_index()
    Z   = linkage(matrice_distance, 'ward')
    
    for tt in range(2,nb_cluster_max + 1):
        assignation = fcluster(Z,t=tt,  criterion='distance')
        t_str = 'cluster_' + str(tt)
        data_out[t_str] = assignation
    
    return data_out, Z

In [ ]:
def give_data_cluster(nb_cluster,data_out):
    
    NB_cluster = len(set(data_out['cluster_'+str(nb_cluster)]))
    data_cluster = pd.DataFrame({'cluster':np.arange((NB_cluster)),'line':[[] for i in range(NB_cluster)]})
    
    for j in set(data_out['cluster_'+str(nb_cluster)]):
        il = list(data_out[data_out['cluster_'+str(nb_cluster)]==j]['line'])
        data_cluster.loc[j-1,'line'].extend(il)
          
    return data_cluster

In [ ]:
def give_data_cluster(nb_cluster,data_out):
    data_cluster = pd.DataFrame({'cluster':np.arange((nb_cluster)),'line':[[] for i in range(nb_cluster)]})
    
    for j in set(data_out['cluster_'+str(nb_cluster)]):
        il = list(data_out[data_out['cluster_'+str(nb_cluster)]==j]['line'])
        data_cluster.loc[j,'line'].extend(il)
          
    return data_cluster

# Suffix Tree

## Load Data

In [26]:
data = load_data()

In [30]:
time_cut       = 4 # seconds
nb_dict        = 6 # number of character in the dictionnary 
nb_max_cluster = 100 # 
data           = clean_z_temporal_cut(data=data,time_cut_s=time_cut)
nb_line        = len(data.index)

## Creation of the tree

In [34]:
lines=list(set(data.line))

In [ ]:
data_line = data[data['line']==lines[0]].reset_index()
tree= Tree({lines[0] + ' 0':data_line['seq_6'][0]})

for number in data_line.index[1:]:
    tree.add(lines[0] + ' '+str(number),data_line['seq_6'][number])

for line in lines[:]:
    if lines.index(line)%100==0:
        print(lines.index(line))
    data_line = data[data['line']==line].reset_index()

    for number in data_line.index:
        tree.add(line + ' '+str(number),data_line['seq_6'][number])

In [35]:
Number_larva = pd.Series(np.zeros(len(lines)),index=lines)
for line in lines:
    Number_larva.loc[line] = len(data[data.loc[:,'line']==line])

## Give node liste

In [ ]:
Node = give_node_liste(tree,data,lines)

# Clustering

## Creation vector weight

### tf - idf, frequency–inverse document frequency

Each line $l$ is considered to be a vector in the M-dimensional term space

$ \vec{l} = \{\omega(1,l),\omega(2,l),...,\omega(M,l)\} $ 

where $\omega(i,l)$, is the term $tf-idf$ (frequency–inverse document frequency). 

<font color='blue'>$ \omega(i,l) = (1 +\log tf(i,l)\log(1+N/df(i))$</font> 
with $N$ the number of lines, the term $tf(i,l)$ is the frequency of the $ith$ term in the line $l$ and $df(i)$ is the number of line containing the $ith$ term.

$tf(i,l) = \frac{n_i}{N_l}$, $n_i$ the number of larvae that pass through this node and $N_l$ the number of total larva in the line $l$.

In [ ]:
Vector_weight = pd.DataFrame(np.zeros((len(lines))),index=lines)
for line1 in lines:
    idx = int(lines.index(line1))
    for node in Node:
# We keep only the sequences that appear at a frequency higher than 0.03
        if node[1][idx]/Number_larva.loc[line1]>0.03:
            Vector_weight.loc[line1,str(node[0])] = (np.log(1+node[1][idx]/Number_larva.loc[line1]))*np.log(1+len(lines)/node[2])   
Vector_weight = Vector_weight.fillna(0.0)


### Matrix distance

In [ ]:
Mdistance = squareform(pdist(Vector_weight, 'cosine'))
Mdistance = np.nan_to_num(Mdistance)

In [ ]:
Dmdistance = pd.DataFrame(Mdistance, index = Vector_weight.index, columns = Vector_weight.index)
Dmdistance = Dmdistance.sort_index(axis='columns').sort_index(axis='index')

## Hierarchical clustering

In [ ]:
data_out, Z = hierarchical_clusters(data = data, nb_dict=nb_dict, matrice_distance=Dmdistance, nb_cluster_max=20)

## Choose the number of cluster

In [38]:
def sil_scores(X, Z, ts):
    '''
    Creates a list of silhouette scores for a predetermined range of clusters.
    '''
    scores = []
    
    for num_clust in ts:
        scores.append(silhouette_score(X, fcluster(Z, t=num_clust, criterion='distance')))
    
    return scores

In [ ]:
scores = sil_scores(Dmdistance,Z,range(2,20))
for t, s in zip(range(2,50), scores):
    print(f"With {t} clusters, the silhouette score is {s}")

We choose a the distance which maximize the sihoulette score and maximises the number of clusters

In [ ]:
distance_cluster = 8

In [ ]:
data_cluster = give_data_cluster(distance_cluster,data_out)

## Plot the clustering

In [ ]:
labels_liste = ['']*len(Dmdistance.index)

In [ ]:
fig, axes = plt.subplots(figsize=(21, 3))
dn  = dendrogram(Z, color_threshold=distance_cluster, labels=labels_liste, orientation="top", ax=axes)

# Show sequences

In [ ]:
Vector_prob = pd.DataFrame(np.zeros((len(LINES))),index=LINES)
for line1 in LINES:
    idx = int(LINES.index(line1))
    for node in Node:
# We keep only the sequences that appear at a frequency higher than 0.03
        if node[1][idx]/Number_larva.loc[line1]>0.03:
            Vector_prob.loc[line1,str(node[0])] = node[1][idx]/Number_larva.loc[line1]
Vector_prob = Vector_prob.fillna(0.0) 

In [ ]:
Panda_Weight = show_weight_sequences(data_cluster, data_out, Vector_prob, 6, mode='motif')